# MLP Bag of Words

En esta notebook, haremos un clasificador en base a una MLP + Bag of Words



In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import random
import torch

torch.manual_seed(2019)
np.random.seed(2019)
tf.random.set_random_seed(2019)
random.seed(2019)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/reference_es.tsv", header=None, 
                        names=["text", "HS", "TR", "AG"], quoting=csv.QUOTE_NONE)


text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]
text_test, y_test = df_test["text"], df_test["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))
print("Instancias de test: {}".format(len(df_test)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500
Instancias de test: 1600


In [2]:
from hate.nn.preprocessing import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tok_args = {
    "preserve_case": False,
    "deaccent": False,
    "reduce_len": True,
    "strip_handles": False,
    "alpha_only": False,
    "stem": True
}

tokenizer = Tokenizer(**tok_args)


vect = CountVectorizer(tokenizer=tokenizer.tokenize, 
                       max_df=0.65, min_df=0.00075, ngram_range=(1, 2), binary=True)

vect.fit(df_train["text"])

bow_train = vect.transform(df_train["text"])
bow_dev = vect.transform(df_dev["text"])
bow_test = vect.transform(df_test["text"])
bow_train.shape, bow_dev.shape, bow_dev.shape

Using TensorFlow backend.


((4500, 5649), (500, 5649), (500, 5649))

In [7]:
X_train, y_train = df_train["text"], df_train["HS"]
X_dev, y_dev = df_dev["text"], df_dev["HS"]
X_test, y_test = df_test["text"], df_test["HS"]

X_train.shape, y_train.shape

((4500,), (4500,))

In [6]:
from hate.nn import BowModel
from keras.optimizers import Adam

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01,
}

vect_args = {
    "max_df": 0.65, 
    "min_df": 0.00075,
    "ngram_range": (1, 2),
    "binary": True
}

model = BowModel(num_words=5500, 
    tokenize_args=tok_args, vectorize_args=vect_args,
    dense_units=[512, 128], dropout=[0.75, 0.50]
)

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=5, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/5
4500/4500 [==============================] - 2s 339us/step - loss: 0.6230 - acc: 0.6596 - val_loss: 0.5420 - val_acc: 0.7420
Epoch 2/5
4500/4500 [==============================] - 1s 246us/step - loss: 0.4609 - acc: 0.8016 - val_loss: 0.4582 - val_acc: 0.7920
Epoch 3/5
4500/4500 [==============================] - 1s 247us/step - loss: 0.3570 - acc: 0.8573 - val_loss: 0.4474 - val_acc: 0.8020
Epoch 4/5
4500/4500 [==============================] - 1s 250us/step - loss: 0.2981 - acc: 0.8869 - val_loss: 0.4464 - val_acc: 0.8080
Epoch 5/5
4500/4500 [==============================] - 1s 241us/step - loss: 0.2482 - acc: 0.9073 - val_loss: 0.4601 - val_acc: 0.7980


In [8]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)

Evaluación sobre dev
500/500 [==============================] - 0s 100us/step
Loss           : 0.4601
Accuracy       : 0.7980
Precision(1)   : 0.7895
Precision(1)   : 0.8041
Precision(avg) : 0.7968

Recall(1)      : 0.7432
Recall(0)      : 0.8417
Recall(avg)    : 0.7925

F1(1)          : 0.7657
F1(0)          : 0.8225
F1(avg)        : 0.7941


Evaluación sobre test
1600/1600 [==============================] - 0s 87us/step
Loss           : 0.6549
Accuracy       : 0.7100
Precision(1)   : 0.6376
Precision(1)   : 0.7680
Precision(avg) : 0.7028

Recall(1)      : 0.6879
Recall(0)      : 0.7255
Recall(avg)    : 0.7067

F1(1)          : 0.6618
F1(0)          : 0.7462
F1(avg)        : 0.7040


## Error Analysis

In [9]:
df_dev["proba"] = model.predict(X_dev)


true_positives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] >= 0.5)].copy()
true_negatives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] < 0.5)].copy()

false_positives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] > 0.5)].copy()
false_positives.sort_values("proba", ascending=False, inplace=True)


false_negatives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] < 0.5)].copy()
false_negatives.sort_values("proba", ascending=True, inplace=True)

conf_matrix = pd.DataFrame([
    {"real":"hs=1", "pred_true": len(true_positives), "pred_false": len(false_negatives)},
    {"real":"hs=0", "pred_true": len(false_positives), "pred_false": len(true_negatives)}
])



conf_matrix.set_index("real", inplace=True)

print("Falsos negativos: {}".format(len(false_negatives)))
print("Falsos positivos: {}".format(len(false_positives)))

conf_matrix[["pred_true", "pred_false"]]

Falsos negativos: 57
Falsos positivos: 44


,pred_true,pred_false
real,,
hs=1,165,57
hs=0,44,234


## Falsos Negativos


In [10]:
pd.set_option('max_colwidth', 300)

false_negatives[["text", "proba"]]


,text,proba
id,,
21072,"Voy andando por la calle y me han dicho cosas en árabe ya que me estas acosando dime lo en español para que te entienda, asique como yo no les entiendo me he cagado en su puta madre en árabe para que ellos a mi si.",0.004687
20711,@lopezdoriga eso es culpa de los gobiernos corruptos que tienen arreglos con esta escoria. Ojalá se muera la perra esa...,0.010608
24925,"La verdad que si, no veia la hora de llegar a los 5 mil amigos😂 ahora de a poco voy borrando esos indios indocumentados😂 https://t.co/Xw8q4GLM8v",0.027638
21727,"España crea su primer 'campo de refugiados' en la bahía de Algeciras https://t.co/l7Zmh2kvPN vía @elmundoes Gran noticia para el contribuyente español. Españoles a pagar, a pagar. Las pensiones pensad en las pensiones. Esto la sanidad como es gratis podemos dejar de pagarla no?",0.029632
21152,@NatyBurgos15 @Horaciogenta @PaolaPa05 Cállate vos Black and bitch como la puta falsa abogada. Espero se pudra en la cárcel y devuelva todo. Dedícate a estudiar en serio. La droga es mala. Cristina puta,0.031110
21215,"Que tal otro aporte anónimo son de la puta de mi esposa, así los dejó en navidad enseñando le los calzones al vecino rt y siguenos @TANGASUCIA @Tangasrobadas https://t.co/70ioUjveqY",0.034408
23716,"- El otro día me comí a Laura es muy fácil que guarra jajajaja - Laura también te comió a ti entonces tú eres igual de guarro e igual de fácil. - Puto feminazi putas modas eres feminista solo para ligar bien queda gilipollas - em, ok",0.035447
22065,| lo único que dominas es mi polla en tu cara. https://t.co/qlYOd4zPwW,0.053814
22199,Todo aquél que se haya molestado en investigar un poco sabe que desde tiempos antiguos los mayores negreros y esclavistas del mundo fueron los árabes. https://t.co/DY5Uo8979v,0.054547


¿Cómo los tokenizamos?

# Conclusiones

- Modelo BoW con una MLP anda bastante bien

No estarían convenciendo las traducciones...

Diría que intentemos un ensemble con caracteres o n-gramas.